In [1]:
"""
Random Forest
Global Feature Selection 
    - X = 'Depth (m)','iso_classes_encoded', 'Oberhollenzer_classes', 
    'σ,v (kPa)', 'σ',v (kPa)'.
    
    - Y = 'qc (MPa)','fs (kPa)'.

Link: https://github.com/DowellChan/ResNetRegression
"""
#General Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_squared_log_error
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler, CSVLogger
from sklearn.model_selection import train_test_split
import time

'''
PreProcessing
''' 
#Loading Database
df_cpt=pd.read_csv(r'/content/drive/MyDrive/Colab Notebooks/Target_Encoded_Database.csv')

#Transforming to DataFrame
df_cpt = pd.DataFrame(df_cpt)
df_cpt = df_cpt.drop(columns='Unnamed: 0')

#Dropping zeros from qc
df_cpt = df_cpt[~(df_cpt[['qc (MPa)']] == 0).any(axis = 1)]
df_cpt = df_cpt[~(df_cpt[['qc (MPa)']] < 0).any(axis = 1)]
df_cpt = df_cpt[~(df_cpt[['SBT (-)']] == 0).any(axis = 1)]
df_cpt = df_cpt[~(df_cpt[['SBTn (-)']] == 0).any(axis = 1)]
df_cpt = df_cpt[~(df_cpt[['Mod. SBTn (-)']] == 0).any(axis = 1)]

#Transforming to NumpyArray
df_cpt = df_cpt.to_numpy()

'''
Inputs
'''
X = df_cpt[:, [11, 17, 18, 19, 27]]

#1--> 'Depth (m)' --> index: 1
#2 --> 'σ,v (kPa)' --> index: 9
#3 --> "σ',v (kPa)" --> index: 11
#4 --> 'SBT (-)' --> index: 17
#5 --> 'SBTn (-)' --> index: 18
#6 --> 'Mod. SBTn (-)' --> index: 19 
#7 --> 'Ic (-)' --> index: 21
#8 --> 'Ic SBT (-)' --> index:22 
#9 --> 'Ib (-)' --> index: 23
#10 --> 'Oberhollenzer_classes' --> index: 24
#11 --> 'iso_classes_encoded' --> index: 27

#X=X.to(device) #To GPU

'''
Outputs
'''
y = df_cpt[:, [2, 3]] 

#1 --> 'qc (MPa)' --> index: 2
#2 --> 'fs (kPa)' --> index: 3

#y=y.to(device) #To GPU

#Scaling the and preparing the data
#scalerX = MinMaxScaler()
#scalerY = MinMaxScaler()

#Scaling the Data
#scaledX=scalerX.fit_transform(X)
#scaledY=scalerY.fit_transform(y)

#Train & Test split
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                   test_size = 0.20)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#CrossVal
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
#Random forest Model
model = RandomForestRegressor(bootstrap = 'True', n_estimators = 1400, max_depth = 300,
                              min_samples_split = 15, min_samples_leaf = 2)

#K-Fold
cv = 9

In [4]:
from sklearn.model_selection import cross_validate
#Initial time
start_time = time.time()

#Evaluating the model
#scores = cross_val_score(model, X_train, y_train, cv = cv, scoring = ['neg_mean_squared_error', 'r2_score'])

scores = cross_validate(model, X_train, y_train, cv = cv, scoring = ('r2', 'neg_mean_squared_error'), return_train_score = True)

#Final time
end_time = (time.time() - start_time)
print('The running time is: %.2f minutes' %(end_time/60))

The running time is: 382.38 minutes


In [5]:
print(scores['train_r2'])

[0.91062468 0.91073344 0.91116452 0.91104623 0.91060453 0.91091781
 0.91069477 0.91070203 0.91129114]


In [6]:
print(scores)

{'fit_time': array([2210.02709222, 2179.30846334, 2196.61077666, 2163.85701728,
       2204.84287548, 2204.34755707, 2222.72900653, 2204.46038675,
       2184.49091482]), 'score_time': array([40.23095298, 39.08073854, 40.50504661, 39.35538483, 39.68252993,
       39.81574988, 40.29346633, 39.55293918, 38.97016406]), 'test_r2': array([0.87294766, 0.86991195, 0.86555518, 0.86700358, 0.87090551,
       0.86649222, 0.86923095, 0.87012009, 0.86299673]), 'train_r2': array([0.91062468, 0.91073344, 0.91116452, 0.91104623, 0.91060453,
       0.91091781, 0.91069477, 0.91070203, 0.91129114]), 'test_neg_mean_squared_error': array([-445.97422327, -458.3976855 , -469.30540006, -458.1067383 ,
       -451.3949186 , -449.83238913, -448.77996908, -447.12693216,
       -475.02676023]), 'train_neg_mean_squared_error': array([-300.9663177 , -300.09399829, -299.11340633, -300.19889022,
       -301.15014181, -301.47427012, -301.49843707, -301.23214929,
       -299.16857927])}
